<a href="https://colab.research.google.com/github/Amplil/nlp-introduction/blob/main/10_named_entity_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Named Entity Recognition using Neural Networks

## History

### 2020/9/3

I fixed some issues. Sorry for the inconvenience.

- Change the name of pre-trained BERT model(bert-base-japanese-whole-word-masking -> cl-tohoku/bert-base-japanese-whole-word-masking)
- Update `evaluate` function due to the version upgrade of Transformers(v2.3.0 -> v3.1.0)
- Fix the version of transformers(v3.1.0)
- Reduce the `batch_size` from 32 to 16 due to OOM

## Setup

In [ ]:
%tensorflow_version 2.x

In [ ]:
!pip install seqeval transformers==3.1.0

     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 890kB 36.6MB/s 
     |████████████████████████████████| 1.2MB 48.9MB/s 
     |████████████████████████████████| 3.0MB 35.3MB/s 
     |████████████████████████████████| 901kB 43.7MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=c60d521c1ba1dc5dd51ce2d69d779596eefbb6ff9b9970e77ea9c8bddd03d661
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval


In [ ]:
!mkdir data
!mkdir models
!wget https://raw.githubusercontent.com/Hironsan/IOB2Corpus/master/ja.wikipedia.conll -P data/

--2021-05-19 09:00:48--  https://raw.githubusercontent.com/Hironsan/IOB2Corpus/master/ja.wikipedia.conll
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1297592 (1.2M) [text/plain]
Saving to: ‘data/ja.wikipedia.conll’

ja.wikipedia.conll  100%[===================>]   1.24M  --.-KB/s    in 0.03s   

2021-05-19 09:00:48 (41.1 MB/s) - ‘data/ja.wikipedia.conll’ saved [1297592/1297592]



### Hyper-parameters

In [ ]:
batch_size = 32
epochs = 100
num_words = 15000

### Imports

In [ ]:
import re

import numpy as np
import tensorflow as tf
from seqeval.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, Input, Embedding, LSTM, Bidirectional
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

## The dataset

### Load the [ja.wikipedia.conll](https://github.com/Hironsan/IOB2Corpus)

In [ ]:
def load_dataset(filename, encoding='utf-8'):
    """Loads data and label from a file.
    Args:
        filename (str): path to the file.
        encoding (str): file encoding format.
        The file format is tab-separated values.
        A blank line is required at the end of a sentence.
        For example:
        ```
        EU	B-ORG
        rejects	O
        German	B-MISC
        call	O
        to	O
        boycott	O
        British	B-MISC
        lamb	O
        .	O
        Peter	B-PER
        Blackburn	I-PER
        ...
        ```
    Returns:
        tuple(numpy array, numpy array): data and labels.
    Example:
        >>> filename = 'conll2003/en/ner/train.txt'
        >>> data, labels = load_data_and_labels(filename)
    """
    sents, labels = [], []
    words, tags = [], []
    with open(filename, encoding=encoding) as f:
        for line in f:
            line = line.rstrip()
            if line:
                word, tag = line.split('\t')
                words.append(word)
                tags.append(tag)
            else:
                sents.append(words)
                labels.append(tags)
                words, tags = [], []
        if words:
            sents.append(words)
            labels.append(tags)

    return sents, labels

In [ ]:
x, y = load_dataset('./data/ja.wikipedia.conll')

### Preprocess the dataset

In [ ]:
class Vocab:

    def __init__(self, num_words=None, lower=True, oov_token=None):
        self.tokenizer = tf.keras.preprocessing.text.Tokenizer(
            num_words=num_words,
            oov_token=oov_token,
            filters='',
            lower=lower,
            split='\t'
        )

    def fit(self, sequences):
        texts = self._texts(sequences)
        self.tokenizer.fit_on_texts(texts)
        return self

    def encode(self, sequences):
        texts = self._texts(sequences)
        return self.tokenizer.texts_to_sequences(texts)

    def decode(self, sequences):
        texts = self.tokenizer.sequences_to_texts(sequences)
        return [text.split(' ') for text in texts]

    def _texts(self, sequences):
        return ['\t'.join(words) for words in sequences]

    def get_index(self, word):
        return self.tokenizer.word_index.get(word)

    @property
    def size(self):
        """Return vocabulary size."""
        return len(self.tokenizer.word_index) + 1

    def save(self, file_path):
        with open(file_path, 'w') as f:
            config = self.tokenizer.to_json()
            f.write(config)

    @classmethod
    def load(cls, file_path):
        with open(file_path) as f:
            tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(f.read())
            vocab = cls()
            vocab.tokenizer = tokenizer
        return vocab


def normalize_number(text, reduce=True):
    if reduce:
        normalized_text = re.sub(r'\d+', '0', text)
    else:
        normalized_text = re.sub(r'\d', '0', text)
    return normalized_text


def preprocess_dataset(sequences):
    sequences = [[normalize_number(w) for w in words] for words in sequences]
    return sequences


def create_dataset(sequences, vocab):
    sequences = vocab.encode(sequences)
    sequences = pad_sequences(sequences, padding='post')
    return sequences

In [ ]:
x = preprocess_dataset(x)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
source_vocab = Vocab(num_words=num_words, oov_token='<UNK>').fit(x_train)
target_vocab = Vocab(lower=False).fit(y_train)
x_train = create_dataset(x_train, source_vocab)
y_train = create_dataset(y_train, target_vocab)

## The models

### Build the models

In [ ]:
class UnidirectionalModel:

    def __init__(self, input_dim, output_dim, emb_dim=100, hid_dim=100, embeddings=None):
        self.input = Input(shape=(None,), name='input')
        if embeddings is None:
            self.embedding = Embedding(input_dim=input_dim,
                                       output_dim=emb_dim,
                                       mask_zero=True,
                                       name='embedding')
        else:
            self.embedding = Embedding(input_dim=embeddings.shape[0],
                                       output_dim=embeddings.shape[1],
                                       mask_zero=True,
                                       weights=[embeddings],
                                       name='embedding')
        self.lstm = LSTM(hid_dim,
                         return_sequences=True,
                         name='lstm')
        self.fc = Dense(output_dim, activation='softmax')

    def build(self):
        x = self.input
        embedding = self.embedding(x)
        lstm = self.lstm(embedding)
        y = self.fc(lstm)
        return Model(inputs=x, outputs=y)


class BidirectionalModel:

    def __init__(self, input_dim, output_dim, emb_dim=100, hid_dim=100, embeddings=None):
        self.input = Input(shape=(None,), name='input')
        if embeddings is None:
            self.embedding = Embedding(input_dim=input_dim,
                                       output_dim=emb_dim,
                                       mask_zero=True,
                                       name='embedding')
        else:
            self.embedding = Embedding(input_dim=embeddings.shape[0],
                                       output_dim=embeddings.shape[1],
                                       mask_zero=True,
                                       weights=[embeddings],
                                       name='embedding')
        lstm = LSTM(hid_dim,
                    return_sequences=True,
                    name='lstm')
        self.bilstm = Bidirectional(lstm, name='bilstm')
        self.fc = Dense(output_dim, activation='softmax')

    def build(self):
        x = self.input
        embedding = self.embedding(x)
        lstm = self.bilstm(embedding)
        y = self.fc(lstm)
        return Model(inputs=x, outputs=y)

In [ ]:
models = [
    UnidirectionalModel(num_words, target_vocab.size).build(),
    BidirectionalModel(num_words, target_vocab.size).build(),
]

### Train the models

In [ ]:
model_path = 'models/model_{}'
for i, model in enumerate(models):
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

    # Preparing callbacks.
    callbacks = [
        EarlyStopping(patience=3),
        ModelCheckpoint(model_path.format(i), save_best_only=True)
    ]

    # Train the model.
    model.fit(x=x_train,
              y=y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=0.1,
              callbacks=callbacks,
              shuffle=True)

Epoch 1/100
23/23 [==============================] - 36s 103ms/step - loss: 1.8575 - val_loss: 0.7113


INFO:tensorflow:Assets written to: models/model_0/assets


INFO:tensorflow:Assets written to: models/model_0/assets


Epoch 2/100
23/23 [==============================] - 1s 38ms/step - loss: 0.6239 - val_loss: 0.6247


INFO:tensorflow:Assets written to: models/model_0/assets


...

INFO:tensorflow:Assets written to: models/model_1/assets


Epoch 19/100
23/23 [==============================] - 1s 48ms/step - loss: 0.0982 - val_loss: 0.2864
Epoch 20/100
23/23 [==============================] - 1s 41ms/step - loss: 0.0844 - val_loss: 0.2791


INFO:tensorflow:Assets written to: models/model_1/assets


INFO:tensorflow:Assets written to: models/model_1/assets


Epoch 21/100
23/23 [==============================] - 1s 46ms/step - loss: 0.0704 - val_loss: 0.2800
Epoch 22/100
23/23 [==============================] - 1s 42ms/step - loss: 0.0639 - val_loss: 0.2793
Epoch 23/100
23/23 [==============================] - 1s 41ms/step - loss: 0.0518 - val_loss: 0.2859


### Evaluate the models

In [ ]:
class InferenceAPI:
    """A model API that generates output sequence.

    Attributes:
        model: Model.
        source_vocab: source language's vocabulary.
        target_vocab: target language's vocabulary.
    """

    def __init__(self, model, source_vocab, target_vocab):
        self.model = model
        self.source_vocab = source_vocab
        self.target_vocab = target_vocab

    def predict_from_sequences(self, sequences):
        lengths = map(len, sequences)
        sequences = self.source_vocab.encode(sequences)
        sequences = pad_sequences(sequences, padding='post')
        y_pred = self.model.predict(sequences)
        y_pred = np.argmax(y_pred, axis=-1)
        y_pred = self.target_vocab.decode(y_pred)
        y_pred = [y[:l] for y, l in zip(y_pred, lengths)]
        return y_pred

In [ ]:
model_names = ['Unidirectional Model', 'Bidirectional Model']
for i, model_name in enumerate(model_names):
    model = load_model(model_path.format(i))
    api = InferenceAPI(model, source_vocab, target_vocab)
    y_pred = api.predict_from_sequences(x_test)
    y_pred = api.predict_from_sequences(x_test)
    print(model_name)
    print(classification_report(y_test, y_pred, digits=4))
    print()

Unidirectional Model


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

    ARTIFACT     0.1182    0.0844    0.0985       154
        DATE     0.4361    0.8444    0.5751       315
       EVENT     0.0000    0.0000    0.0000        64
    LOCATION     0.6610    0.5152    0.5791       526
       MONEY     0.0000    0.0000    0.0000        12
      NUMBER     0.0836    0.1284    0.1013       218
ORGANIZATION     0.1489    0.1855    0.1652       248
       OTHER     0.0000    0.0000    0.0000        75
     PERCENT     0.0000    0.0000    0.0000        52
      PERSON     0.2222    0.1250    0.1600       224
        TIME     0.0000    0.0000    0.0000         5

   micro avg     0.3399    0.3444    0.3422      1893
   macro avg     0.1518    0.1712    0.1526      1893
weighted avg     0.3213    0.3444    0.3169      1893


Bidirectional Model
              precision    recall  f1-score   support

    ARTIFACT     0.1161    0.0844    0.0977       154
        DATE     0.8121    0.8508    0.8310       315
   

# BERT for Named Entity Recognition

# utils.py

In [ ]:
def evaluate(model, target_vocab, features, labels):
    label_ids = model.predict(features)[0]
    label_ids = np.argmax(label_ids, axis=-1)
    y_pred = [[] for _ in range(label_ids.shape[0])]
    y_true = [[] for _ in range(label_ids.shape[0])]
    for i in range(label_ids.shape[0]):
        for j in range(label_ids.shape[1]):
            if labels[i][j] == 0:
                continue
            y_pred[i].append(label_ids[i][j])
            y_true[i].append(labels[i][j])
    y_pred = target_vocab.decode(y_pred)
    y_true = target_vocab.decode(y_true)
    print(classification_report(y_true, y_pred, digits=4))

# preprocessing.py

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences


def convert_examples_to_features(x, y,
                                 vocab,
                                 max_seq_length,
                                 tokenizer):
    pad_token = 0
    features = {
        'input_ids': [],
        'attention_mask': [],
        'token_type_ids': [],
        'label_ids': []
    }
    for words, labels in zip(x, y):
        tokens = [tokenizer.cls_token]
        label_ids = [pad_token]
        for word, label in zip(words, labels):
            word_tokens = tokenizer.tokenize(word)
            tokens.extend(word_tokens)
            label_id = vocab.get_index(label)
            label_ids.extend([label_id] + [pad_token] * (len(word_tokens) - 1))
        tokens += [tokenizer.sep_token]

        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(input_ids)
        token_type_ids = [pad_token] * max_seq_length

        features['input_ids'].append(input_ids)
        features['attention_mask'].append(attention_mask)
        features['token_type_ids'].append(token_type_ids)
        features['label_ids'].append(label_ids)

    for name in features:
        features[name] = pad_sequences(features[name], padding='post', maxlen=max_seq_length)

    x = [features['input_ids'], features['attention_mask'], features['token_type_ids']]
    y = features['label_ids']
    return x, y

# model.py

In [ ]:
import tensorflow as tf
from transformers import TFBertForTokenClassification, BertConfig


def build_model(pretrained_model_name_or_path, num_labels):
    config = BertConfig.from_pretrained(
        pretrained_model_name_or_path,
        num_labels=num_labels
    )
    model = TFBertForTokenClassification.from_pretrained(
        pretrained_model_name_or_path,
        config=config
    )
    model.layers[-1].activation = tf.keras.activations.softmax
    return model


def loss_func(num_labels):
    loss_fct = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)

    def loss(y_true, y_pred):
        input_mask = tf.not_equal(y_true, 0)
        logits = tf.reshape(y_pred, (-1, num_labels))
        active_loss = tf.reshape(input_mask, (-1,))
        active_logits = tf.boolean_mask(logits, active_loss)
        train_labels = tf.reshape(y_true, (-1,))
        active_labels = tf.boolean_mask(train_labels, active_loss)
        cross_entropy = loss_fct(active_labels, active_logits)
        return cross_entropy
    return loss

# Training

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from transformers import BertJapaneseTokenizer


def main():
    # Set hyper-parameters.
    batch_size = 16
    epochs = 100
    model_path = 'models/'
    pretrained_model_name_or_path = 'cl-tohoku/bert-base-japanese-whole-word-masking'
    maxlen = 250

    # Data loading.
    x, y = load_dataset('./data/ja.wikipedia.conll')
    tokenizer = BertJapaneseTokenizer.from_pretrained(pretrained_model_name_or_path, do_word_tokenize=False)

    # Pre-processing.
    x = preprocess_dataset(x)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    target_vocab = Vocab(lower=False).fit(y_train)
    features_train, labels_train = convert_examples_to_features(
        x_train,
        y_train,
        target_vocab,
        max_seq_length=maxlen,
        tokenizer=tokenizer
    )
    features_test, labels_test = convert_examples_to_features(
        x_test,
        y_test,
        target_vocab,
        max_seq_length=maxlen,
        tokenizer=tokenizer
    )

    # Build model.
    model = build_model(pretrained_model_name_or_path, target_vocab.size)
    model.compile(optimizer='sgd', loss=loss_func(target_vocab.size))

    # Preparing callbacks.
    callbacks = [
        EarlyStopping(patience=3),
    ]

    # Train the model.
    model.fit(x=features_train,
              y=labels_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=0.1,
              callbacks=callbacks,
              shuffle=True)
    model.save_pretrained(model_path)

    # Evaluation.
    evaluate(model, target_vocab, features_test, labels_test)


if __name__ == '__main__':
    main()

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing TFBertForTokenClassification: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForTokenClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier', 'dropout_37']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/100


45/45 [==============================] - 51s 931ms/step - loss: 1.1704 - val_loss: 0.6621
Epoch 2/100
45/45 [==============================] - 39s 867ms/step - loss: 0.6339 - val_loss: 0.5558
Epoch 3/100
45/45 [==============================] - 39s 879ms/step - loss: 0.5297 - val_loss: 0.4998
Epoch 4/100
45/45 [==============================] - 39s 877ms/step - loss: 0.4560 - val_loss: 0.4412
Epoch 5/100
45/45 [==============================] - 39s 872ms/step - loss: 0.4068 - val_loss: 0.3963
Epoch 6/100
45/45 [==============================] - 39s 878ms/step - loss: 0.3710 - val_loss: 0.3706
Epoch 7/100
45/45 [==============================] - 39s 877ms/step - loss: 0.3570 - val_loss: 0.3370
Epoch 8/100
45/45 [==============================] - 39s 875ms/step - loss: 0.3275 - val_loss: 0.3273
Epoch 9/100
45/45 [==============================] - 39s 872ms/step - loss: 0.2795 - val_loss: 0.3030
Epoch 10/100
45/45 [==============================] - 39s 872ms/step - loss: 0.2711 - val_loss

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

    ARTIFACT     0.3077    0.3636    0.3333       154
        DATE     0.7831    0.8254    0.8037       315
       EVENT     0.4085    0.4531    0.4296        64
    LOCATION     0.7767    0.7738    0.7752       526
       MONEY     0.0000    0.0000    0.0000        12
      NUMBER     0.4082    0.5505    0.4688       218
ORGANIZATION     0.5503    0.6613    0.6007       248
       OTHER     0.4500    0.3600    0.4000        75
     PERCENT     0.0000    0.0000    0.0000        52
      PERSON     0.8403    0.8929    0.8658       224
        TIME     0.0000    0.0000    0.0000         5

   micro avg     0.6188    0.6672    0.6421      1893
   macro avg     0.4113    0.4437    0.4252      1893
weighted avg     0.6213    0.6672    0.6418      1893

